# Signal Combiner User Guide

This notebook provides a comprehensive guide to using the `algoshort.combiner` module for combining multiple trading signals into unified trading strategies.

## Table of Contents

1. [Setup and Installation](#1-setup-and-installation)
2. [Understanding Signal Combination](#2-understanding-signal-combination)
3. [HybridSignalCombiner Basics](#3-hybridsignalcombiner-basics)
4. [Entry and Exit Logic](#4-entry-and-exit-logic)
5. [Position Flipping](#5-position-flipping)
6. [Regime Alignment Modes](#6-regime-alignment-modes)
7. [Trade Metadata and Summary](#7-trade-metadata-and-summary)
8. [Grid Search for Signal Combinations](#8-grid-search-for-signal-combinations)
9. [Complete Workflow Example](#9-complete-workflow-example)
10. [Best Practices and Tips](#10-best-practices-and-tips)

## 1. Setup and Installation

First, let's import the required modules.

In [ ]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

# Import combiner classes
from algoshort.combiner import HybridSignalCombiner, SignalGridSearch

# Import regime detection for generating signals
from algoshort.regimes import RegimeDetector
from algoshort.yfinance_handler import YFinanceDataHandler

print("Imports successful!")

## 2. Understanding Signal Combination

### The Problem

Different trading signals capture different market characteristics:

| Signal Type | Captures | Example |
|-------------|----------|--------|
| **Floor/Ceiling** | Long-term trend direction | Market regime (bull/bear) |
| **Moving Average** | Medium-term trend | Trend momentum |
| **Breakout** | Short-term momentum | Entry timing |
| **Turtle Trader** | Dual-timeframe | Entry/exit confirmation |

### The Solution: Hybrid Signal Combination

The `HybridSignalCombiner` uses a three-signal approach:

```
┌─────────────────────────────────────────────────────────────┐
│                    HYBRID SIGNAL LOGIC                       │
├─────────────────────────────────────────────────────────────┤
│                                                              │
│  DIRECTION SIGNAL (e.g., Floor/Ceiling)                      │
│  ├─ +1: Bullish regime → Only allow LONG entries            │
│  ├─  0: Neutral regime → Depends on alignment mode          │
│  └─ -1: Bearish regime → Only allow SHORT entries           │
│                                                              │
│  ENTRY SIGNAL (e.g., Breakout)                               │
│  ├─ +1: Long entry trigger                                   │
│  ├─  0: No entry                                             │
│  └─ -1: Short entry trigger                                  │
│                                                              │
│  EXIT SIGNAL (e.g., MA Crossover)                            │
│  ├─ +1: Exit short / Bullish signal                          │
│  ├─  0: No exit                                              │
│  └─ -1: Exit long / Bearish signal                           │
│                                                              │
└─────────────────────────────────────────────────────────────┘
```

In [ ]:
# Fetch sample data
handler = YFinanceDataHandler()
df = handler.get_ohlc_data('SPY', period='2y', interval='1d')

print(f"Downloaded {len(df)} rows")
df.head()

In [ ]:
# Generate regime signals
detector = RegimeDetector(df, log_level=logging.WARNING)

# Floor/Ceiling for direction
df_signals = detector.floor_ceiling(lvl=1, threshold=1.5)

# Breakout for entry timing
df_signals = RegimeDetector(df_signals, log_level=logging.WARNING).breakout(window=20)

# SMA crossover for exit timing
df_signals = RegimeDetector(df_signals, log_level=logging.WARNING).sma_crossover(short=10, medium=20, long=50)

# Clean up NaN values
df_signals = df_signals.dropna().reset_index(drop=True)

print(f"Signal columns: {[c for c in df_signals.columns if any(x in c for x in ['rg', 'bo_', 'sma_'])]}")
df_signals[['close', 'rg', 'bo_20', 'sma_102050']].tail(10)

## 3. HybridSignalCombiner Basics

### Creating a Combiner

The `HybridSignalCombiner` takes three signal column names:
- `direction_col`: The regime/direction filter (typically Floor/Ceiling)
- `entry_col`: The entry timing signal
- `exit_col`: The exit timing signal

In [ ]:
# Create a HybridSignalCombiner
combiner = HybridSignalCombiner(
    direction_col='rg',        # Floor/Ceiling regime
    entry_col='bo_20',         # 20-day breakout for entries
    exit_col='sma_102050',     # SMA crossover for exits
    verbose=False              # Set True for detailed trade logging
)

print(f"Direction column: {combiner.direction_col}")
print(f"Entry column: {combiner.entry_col}")
print(f"Exit column: {combiner.exit_col}")

In [ ]:
# Validate signals before combining
validation = combiner.validate_signals(df_signals)

print(f"Validation passed: {validation['valid']}")
if validation['errors']:
    print(f"Errors: {validation['errors']}")
if validation['warnings']:
    print(f"Warnings: {validation['warnings']}")

In [ ]:
# Combine signals
df_combined = combiner.combine_signals(
    df_signals.copy(),
    output_col='hybrid_signal',
    allow_flips=True,
    require_regime_alignment=True
)

# View results
print("Signal Distribution:")
print(df_combined['hybrid_signal'].value_counts().sort_index())

df_combined[['close', 'rg', 'bo_20', 'sma_102050', 'hybrid_signal']].tail(10)

## 4. Entry and Exit Logic

### Entry Conditions

**From FLAT position:**
- Enter LONG when: `entry=1` AND `direction=1` (strict mode)
- Enter SHORT when: `entry=-1` AND `direction=-1` (strict mode)

### Exit Conditions

**From LONG position:**
- Exit on `exit=-1` (bearish exit signal)
- Exit on `direction=-1` (regime turns bearish)

**From SHORT position:**
- Exit on `exit=1` (bullish exit signal)
- Exit on `direction=1` (regime turns bullish)

In [ ]:
# Demonstrate entry/exit logic with a simple example
demo_df = pd.DataFrame({
    'direction': [0, 1, 1, 1, 1, 1, 1, -1, -1, -1],
    'entry':     [0, 1, 0, 0, 0, 0, 0, -1, 0, 0],
    'exit':      [0, 0, 0, 0, -1, 0, 0, 0, 0, 1],
})

demo_combiner = HybridSignalCombiner(
    direction_col='direction',
    entry_col='entry',
    exit_col='exit',
    verbose=True  # Enable verbose to see trade logic
)

print("Entry/Exit Logic Demo:")
print("=" * 50)
demo_result = demo_combiner.combine_signals(demo_df, output_col='position', allow_flips=True)
print("\nResult:")
print(demo_result)

## 5. Position Flipping

### With `allow_flips=True`:
- Can flip directly from LONG to SHORT (or vice versa)
- Requires both entry signal AND regime alignment

### With `allow_flips=False`:
- Must exit to FLAT before entering opposite direction
- More conservative approach

In [ ]:
# Compare flip vs no-flip behavior
flip_df = pd.DataFrame({
    'direction': [0, 1, 1, 1, -1, -1, -1, 1, 1, 1],
    'entry':     [0, 1, 0, 0, -1, 0, 0, 1, 0, 0],
    'exit':      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
})

flip_combiner = HybridSignalCombiner(
    direction_col='direction',
    entry_col='entry',
    exit_col='exit'
)

# With flips allowed
result_flip = flip_combiner.combine_signals(
    flip_df.copy(), output_col='with_flip', allow_flips=True
)

# Without flips
result_noflip = flip_combiner.combine_signals(
    flip_df.copy(), output_col='no_flip', allow_flips=False
)

# Compare
comparison = flip_df.copy()
comparison['with_flip'] = result_flip['with_flip']
comparison['no_flip'] = result_noflip['no_flip']

print("Flip vs No-Flip Comparison:")
print(comparison)

In [ ]:
# Visualize the difference
fig, axes = plt.subplots(3, 1, figsize=(12, 8), sharex=True)

# Direction signal
ax1 = axes[0]
ax1.bar(comparison.index, comparison['direction'], color=['green' if x > 0 else 'red' if x < 0 else 'gray' for x in comparison['direction']])
ax1.set_ylabel('Direction')
ax1.set_title('Direction Signal (Regime)')
ax1.axhline(0, color='black', linewidth=0.5)

# With flips
ax2 = axes[1]
ax2.bar(comparison.index, comparison['with_flip'], color=['green' if x > 0 else 'red' if x < 0 else 'gray' for x in comparison['with_flip']])
ax2.set_ylabel('Position')
ax2.set_title('With Flips Allowed (allow_flips=True)')
ax2.axhline(0, color='black', linewidth=0.5)

# Without flips
ax3 = axes[2]
ax3.bar(comparison.index, comparison['no_flip'], color=['green' if x > 0 else 'red' if x < 0 else 'gray' for x in comparison['no_flip']])
ax3.set_ylabel('Position')
ax3.set_xlabel('Bar')
ax3.set_title('Without Flips (allow_flips=False)')
ax3.axhline(0, color='black', linewidth=0.5)

plt.tight_layout()
plt.show()

## 6. Regime Alignment Modes

### Strict Mode (`require_regime_alignment=True`):
- Long entry requires `direction=1`
- Short entry requires `direction=-1`
- Neutral regime (`direction=0`) blocks all entries

### Loose Mode (`require_regime_alignment=False`):
- Long entry allowed if `direction != -1`
- Short entry allowed if `direction != 1`
- Neutral regime allows both directions

In [ ]:
# Compare strict vs loose regime alignment
neutral_df = pd.DataFrame({
    'direction': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],  # All neutral
    'entry':     [0, 1, 0, 0, -1, 0, 1, 0, 0, -1],
    'exit':      [0, 0, 0, -1, 0, 1, 0, -1, 0, 0],
})

neutral_combiner = HybridSignalCombiner(
    direction_col='direction',
    entry_col='entry',
    exit_col='exit'
)

# Strict mode
result_strict = neutral_combiner.combine_signals(
    neutral_df.copy(), output_col='strict', require_regime_alignment=True
)

# Loose mode
result_loose = neutral_combiner.combine_signals(
    neutral_df.copy(), output_col='loose', require_regime_alignment=False
)

compare_df = neutral_df.copy()
compare_df['strict'] = result_strict['strict']
compare_df['loose'] = result_loose['loose']

print("Strict vs Loose Regime Alignment (with neutral regime):")
print(compare_df)

print(f"\nStrict mode - Total position changes: {(compare_df['strict'] != compare_df['strict'].shift()).sum()}")
print(f"Loose mode - Total position changes: {(compare_df['loose'] != compare_df['loose'].shift()).sum()}")

## 7. Trade Metadata and Summary

### Adding Trade Metadata

The `add_signal_metadata()` method adds detailed columns:
- `position_changed`: Boolean indicating position change
- `trade_type`: Type of trade action
- `bars_in_position`: Running count of bars held
- `position_direction`: Current position as text

In [ ]:
# Add metadata to the combined signals
df_with_meta = combiner.add_signal_metadata(df_combined.copy(), output_col='hybrid_signal')

print("Metadata columns added:")
print([c for c in df_with_meta.columns if c in ['position_changed', 'trade_type', 'bars_in_position', 'position_direction']])

# View some position changes
position_changes = df_with_meta[df_with_meta['position_changed'] == True]
print(f"\nTotal position changes: {len(position_changes)}")
print(f"\nTrade type distribution:")
print(df_with_meta['trade_type'].value_counts())

In [ ]:
# Get comprehensive trade summary
summary = combiner.get_trade_summary(df_combined, output_col='hybrid_signal')

print("Trade Summary:")
print("=" * 50)
print(f"\nPosition Distribution:")
print(f"  Total bars: {summary['total_bars']}")
print(f"  Long bars: {summary['long_bars']} ({summary['long_pct']:.1f}%)")
print(f"  Short bars: {summary['short_bars']} ({summary['short_pct']:.1f}%)")
print(f"  Flat bars: {summary['flat_bars']} ({summary['flat_pct']:.1f}%)")

print(f"\nTrade Counts:")
print(f"  Long entries: {summary['entry_long_count']}")
print(f"  Short entries: {summary['entry_short_count']}")
print(f"  Long exits: {summary['exit_long_count']}")
print(f"  Short exits: {summary['exit_short_count']}")
print(f"  Long→Short flips: {summary['flip_long_to_short_count']}")
print(f"  Short→Long flips: {summary['flip_short_to_long_count']}")

print(f"\nTotal entries: {summary['total_entries']}")
print(f"Total exits: {summary['total_exits']}")

print(f"\nAverage Holding Period:")
print(f"  Avg bars per long trade: {summary['avg_bars_per_long_trade']:.1f}")
print(f"  Avg bars per short trade: {summary['avg_bars_per_short_trade']:.1f}")

## 8. Grid Search for Signal Combinations

The `SignalGridSearch` class allows testing all combinations of entry and exit signals to find the best performing strategies.

### How It Works:
1. Fixed direction signal (e.g., Floor/Ceiling)
2. Test all combinations of entry/exit signals
3. Compare trade statistics
4. Filter and rank results

In [ ]:
# Prepare signals for grid search
# We need more signal columns
handler = YFinanceDataHandler()
df_grid = handler.get_ohlc_data('SPY', period='2y', interval='1d')

# Generate multiple signals
detector = RegimeDetector(df_grid, log_level=logging.WARNING)

# Direction signal (Floor/Ceiling)
df_grid = detector.floor_ceiling(lvl=1, threshold=1.5)

# Entry/Exit signals
detector2 = RegimeDetector(df_grid, log_level=logging.WARNING)
df_grid = detector2.breakout(window=20)
df_grid = RegimeDetector(df_grid, log_level=logging.WARNING).turtle(slow=50, fast=20)
df_grid = RegimeDetector(df_grid, log_level=logging.WARNING).sma_crossover(short=10, medium=20, long=50)
df_grid = RegimeDetector(df_grid, log_level=logging.WARNING).ema_crossover(short=12, medium=26, long=50)

# Clean up
df_grid = df_grid.dropna().reset_index(drop=True)

print(f"DataFrame shape: {df_grid.shape}")
print(f"\nAvailable signal columns:")
signal_cols = [c for c in df_grid.columns if any(x in c for x in ['rg', 'bo_', 'tt_', 'sma_', 'ema_'])]
for col in signal_cols:
    print(f"  - {col}")

In [ ]:
# Create grid search
entry_exit_signals = ['bo_20', 'tt_5020', 'sma_102050', 'ema_122650']

searcher = SignalGridSearch(
    df=df_grid,
    available_signals=entry_exit_signals,
    direction_col='rg'  # Floor/Ceiling direction
)

In [ ]:
# Generate the grid (preview)
grid = searcher.generate_grid()

print(f"\nFirst 5 combinations:")
for combo in grid[:5]:
    print(f"  Entry: {combo['entry']:15s} Exit: {combo['exit']:15s} Name: {combo['name']}")

In [ ]:
# Run grid search
results = searcher.run_grid_search(
    allow_flips=True,
    require_regime_alignment=True,
    verbose=False
)

print(f"\nResults shape: {results.shape}")
results.head()

In [ ]:
# Get results summary
summary = searcher.get_results_summary()

print("Grid Search Summary:")
print("=" * 50)
for key, value in summary.items():
    print(f"{key}: {value}")

In [ ]:
# Filter combinations
filtered = searcher.filter_combinations(
    min_trades=5,      # At least 5 trades
    max_flat_pct=80    # No more than 80% flat
)

print(f"\nFiltered results (min 5 trades, max 80% flat):")
filtered[['combination_name', 'total_trades', 'long_pct', 'short_pct', 'flat_pct']].sort_values('total_trades', ascending=False)

In [ ]:
# Visualize results
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Total trades by combination
ax1 = axes[0, 0]
results_sorted = results.sort_values('total_trades', ascending=True)
ax1.barh(results_sorted['combination_name'], results_sorted['total_trades'])
ax1.set_xlabel('Total Trades')
ax1.set_title('Total Trades by Combination')

# Position distribution
ax2 = axes[0, 1]
x = np.arange(len(results))
width = 0.25
ax2.bar(x - width, results['long_pct'], width, label='Long %', color='green')
ax2.bar(x, results['short_pct'], width, label='Short %', color='red')
ax2.bar(x + width, results['flat_pct'], width, label='Flat %', color='gray')
ax2.set_xticks(x)
ax2.set_xticklabels(results['combination_name'], rotation=45, ha='right')
ax2.set_ylabel('Percentage')
ax2.set_title('Position Distribution by Combination')
ax2.legend()

# Trade types
ax3 = axes[1, 0]
results_sample = results.head(8)  # First 8 for readability
x = np.arange(len(results_sample))
ax3.bar(x - 0.2, results_sample['long_trades'], 0.4, label='Long Entries', color='green')
ax3.bar(x + 0.2, results_sample['short_trades'], 0.4, label='Short Entries', color='red')
ax3.set_xticks(x)
ax3.set_xticklabels(results_sample['combination_name'], rotation=45, ha='right')
ax3.set_ylabel('Count')
ax3.set_title('Long vs Short Entries')
ax3.legend()

# Flip counts
ax4 = axes[1, 1]
flip_data = results[['combination_name', 'long_to_short_flips', 'short_to_long_flips']]
flip_data['total_flips'] = flip_data['long_to_short_flips'] + flip_data['short_to_long_flips']
flip_sorted = flip_data.sort_values('total_flips', ascending=True)
ax4.barh(flip_sorted['combination_name'], flip_sorted['total_flips'])
ax4.set_xlabel('Total Flips')
ax4.set_title('Position Flips by Combination')

plt.tight_layout()
plt.show()

## 9. Complete Workflow Example

Let's put everything together in a complete workflow.

In [ ]:
# Complete workflow: From data to trading signals
from algoshort import (
    YFinanceDataHandler,
    RegimeDetector,
    StopLossCalculator,
    ReturnsCalculator
)
from algoshort.combiner import HybridSignalCombiner

# Step 1: Get data
handler = YFinanceDataHandler()
symbol = 'AAPL'
df = handler.get_ohlc_data(symbol, period='2y', interval='1d')
print(f"Step 1: Downloaded {len(df)} rows for {symbol}")

In [ ]:
# Step 2: Generate direction signal (Floor/Ceiling)
detector = RegimeDetector(df, log_level=logging.WARNING)
df = detector.floor_ceiling(lvl=1, threshold=1.5)
print(f"Step 2: Floor/Ceiling signal generated")
print(f"  Regime distribution: {df['rg'].value_counts().to_dict()}")

In [ ]:
# Step 3: Generate entry and exit signals
df = RegimeDetector(df, log_level=logging.WARNING).breakout(window=20)
df = RegimeDetector(df, log_level=logging.WARNING).sma_crossover(short=10, medium=20, long=50)
df = df.dropna().reset_index(drop=True)
print(f"Step 3: Entry (breakout) and exit (SMA) signals generated")

In [ ]:
# Step 4: Combine signals
combiner = HybridSignalCombiner(
    direction_col='rg',
    entry_col='bo_20',
    exit_col='sma_102050'
)

df = combiner.combine_signals(
    df,
    output_col='strategy_signal',
    allow_flips=True,
    require_regime_alignment=True
)

summary = combiner.get_trade_summary(df, output_col='strategy_signal')
print(f"Step 4: Signals combined")
print(f"  Total trades: {summary['total_entries']}")
print(f"  Position distribution: Long {summary['long_pct']:.1f}%, Short {summary['short_pct']:.1f}%, Flat {summary['flat_pct']:.1f}%")

In [ ]:
# Step 5: Calculate stop-loss
stop_calc = StopLossCalculator(df)
df = stop_calc.atr_stop_loss(
    signal='strategy_signal',
    multiplier=2.0,
    window=14,
    forward_fill=True
)
print(f"Step 5: ATR stop-loss calculated")

In [ ]:
# Step 6: Calculate strategy returns
df['daily_return'] = df['close'].pct_change()
df['strategy_return'] = df['strategy_signal'].shift(1) * df['daily_return']
df['cumulative_return'] = (1 + df['strategy_return'].fillna(0)).cumprod()
df['buy_hold_return'] = (1 + df['daily_return'].fillna(0)).cumprod()

print(f"Step 6: Returns calculated")
print(f"  Strategy total return: {(df['cumulative_return'].iloc[-1] - 1) * 100:.2f}%")
print(f"  Buy & Hold return: {(df['buy_hold_return'].iloc[-1] - 1) * 100:.2f}%")

In [ ]:
# Final visualization
fig, axes = plt.subplots(4, 1, figsize=(14, 14), sharex=True)

# Plot 1: Price with regime coloring
ax1 = axes[0]
ax1.plot(df.index, df['close'], 'b-', linewidth=1, label='Close')
signal = df['rg']
ax1.fill_between(df.index, df['close'].min(), df['close'].max(),
                 where=signal > 0, alpha=0.2, color='green', label='Bullish Regime')
ax1.fill_between(df.index, df['close'].min(), df['close'].max(),
                 where=signal < 0, alpha=0.2, color='red', label='Bearish Regime')
ax1.set_ylabel('Price ($)')
ax1.set_title(f'{symbol} - Price with Floor/Ceiling Regime')
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)

# Plot 2: Combined strategy signal
ax2 = axes[1]
signal = df['strategy_signal']
ax2.fill_between(df.index, 0, signal, where=signal > 0, 
                 color='green', alpha=0.7, label='Long')
ax2.fill_between(df.index, 0, signal, where=signal < 0, 
                 color='red', alpha=0.7, label='Short')
ax2.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
ax2.set_ylabel('Position')
ax2.set_title('Combined Strategy Signal')
ax2.legend(loc='upper left')
ax2.set_ylim(-1.5, 1.5)

# Plot 3: Cumulative returns
ax3 = axes[2]
ax3.plot(df.index, df['cumulative_return'], 'g-', linewidth=2, label='Strategy')
ax3.plot(df.index, df['buy_hold_return'], 'b--', linewidth=1, label='Buy & Hold')
ax3.axhline(y=1, color='gray', linestyle='--', alpha=0.5)
ax3.set_ylabel('Cumulative Return')
ax3.set_title('Strategy vs Buy & Hold Returns')
ax3.legend(loc='upper left')
ax3.grid(True, alpha=0.3)

# Plot 4: Drawdown
ax4 = axes[3]
strategy_dd = (df['cumulative_return'] / df['cumulative_return'].expanding().max() - 1) * 100
bh_dd = (df['buy_hold_return'] / df['buy_hold_return'].expanding().max() - 1) * 100
ax4.fill_between(df.index, 0, strategy_dd, alpha=0.5, color='green', label='Strategy DD')
ax4.fill_between(df.index, 0, bh_dd, alpha=0.3, color='blue', label='Buy & Hold DD')
ax4.set_ylabel('Drawdown (%)')
ax4.set_xlabel('Period')
ax4.set_title('Drawdown Comparison')
ax4.legend(loc='lower left')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print final statistics
print(f"\n{'='*60}")
print(f"FINAL PERFORMANCE SUMMARY - {symbol}")
print(f"{'='*60}")
print(f"Strategy Return: {(df['cumulative_return'].iloc[-1] - 1) * 100:+.2f}%")
print(f"Buy & Hold Return: {(df['buy_hold_return'].iloc[-1] - 1) * 100:+.2f}%")
print(f"Strategy Max Drawdown: {strategy_dd.min():.2f}%")
print(f"Buy & Hold Max Drawdown: {bh_dd.min():.2f}%")
print(f"Total Trades: {summary['total_entries']}")

## 10. Best Practices and Tips

### Signal Selection Guidelines

| Signal Role | Recommended Methods | Rationale |
|-------------|---------------------|----------|
| **Direction** | Floor/Ceiling | Captures long-term regime |
| **Entry** | Breakout, Turtle | Captures momentum timing |
| **Exit** | MA Crossover | Smooth trend following |

### Common Pitfalls

1. **Over-trading**: Too many signals → high transaction costs
2. **Regime mismatch**: Entering against regime direction
3. **Late exits**: Using slow signals for exits
4. **No stops**: Always use stop-loss protection

### Performance Tips

In [ ]:
# Tip 1: Use parallel grid search for large parameter spaces
# searcher.run_grid_search_parallel(n_jobs=-1)  # Uses all CPU cores

# Tip 2: Filter results early to focus on viable combinations
# filtered = searcher.filter_combinations(min_trades=10, max_flat_pct=70)

# Tip 3: Export results for further analysis
# searcher.export_results('grid_search_results.csv')
# searcher.export_dataframe('signals_dataframe.csv')

print("Best practices demonstrated!")

### Quick Reference

```python
# Import
from algoshort.combiner import HybridSignalCombiner, SignalGridSearch

# Basic combination
combiner = HybridSignalCombiner(
    direction_col='rg',      # Floor/Ceiling
    entry_col='bo_20',       # Breakout
    exit_col='sma_102050'    # SMA crossover
)
df = combiner.combine_signals(df, output_col='signal')
summary = combiner.get_trade_summary(df, output_col='signal')

# Grid search
searcher = SignalGridSearch(
    df=df,
    available_signals=['bo_20', 'tt_5020', 'sma_102050'],
    direction_col='rg'
)
results = searcher.run_grid_search(allow_flips=True)
filtered = searcher.filter_combinations(min_trades=10)
```

---

## Summary

This guide covered:

1. **Signal Combination Concept**: Using direction + entry + exit signals
2. **HybridSignalCombiner**: Basic usage and configuration
3. **Entry/Exit Logic**: How positions are opened and closed
4. **Position Flipping**: Direct flip vs exit-first approach
5. **Regime Alignment**: Strict vs loose mode
6. **Trade Metadata**: Detailed trade tracking
7. **Grid Search**: Testing all signal combinations
8. **Complete Workflow**: From data to trading strategy

For more details, refer to:
- Test suite: `tests/test_combiner.py`
- Source code: `algoshort/combiner.py`